%%

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import g_key

region<br>
natively display more info in dfs in pycharm

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# endregion

egion<br>
def spaceless_lowers(dataframe):<br>
#     
<br>
#     Params:<br>
#         dataframe: a dataframe with columns that have spaces and uppercase letters<br>
#     Returns:<br>
#         a dataframe with the spaces replaced with _ and all caps made lowercase.<br>
    
<br>
    try:<br>
        cols = dataframe.columns<br>
        cols = [col.replace(' ', '_').lower() for col in cols]<br>
        dataframe.columns = cols<br>
<br>
        return dataframe<br>
<br>
    except NameError:<br>
        print('There is an unresolved reference to the dataframe in the function\'s argument.\n'<br>
              'Make sure that the dataframe has been read and defined.')<br>
<br>
<br>
def find_cities():<br>
    for r, d, files in os.walk('.'):<br>
        for name in files:<br>
            if name == 'cities.csv':<br>
                data = os.path.join(r, name)<br>
                return data<br>
<br>
<br>
def k_to_f(temp):<br>
    temp = round((temp - 273.15) * 9 / 5 + 32, 2)<br>
    return temp<br>
<br>
<br>
def kmh_to_mph(speed):<br>
    return speed * 0.621371<br>
<br>
<br>
def north_or_south(latitude):<br>
    return latitude < 0<br>


In [ ]:
df = pd.read_csv(r'./WeatherPy/cities_2.csv')

In [ ]:
df.rename(columns={'Unnamed: 0': 'city'}, inplace=True)
df.head()

In [ ]:
gmaps.configure(g_key)

In [ ]:
locs = df[['lat', 'long']]
humidity = df.humidity

In [ ]:
fig_layout = {
        'width':'1200px',
        'height':'600px',
        'border':'2px solid black',
        'padding':'2px',
        'margin':'0 auto 0 auto'
}

In [ ]:
fig = gmaps.figure(layout=fig_layout)

In [ ]:
heatmap = gmaps.heatmap_layer(
        locs,
        weights=humidity,
        max_intensity=df.humidity.max(),
        dissipating=False,
        point_radius=5
)

In [ ]:
fig.add_layer(heatmap)

select decent weather

In [ ]:
temp_mask = df['max_temp'].between(70, 90)
humid_mask = df['humidity'].between(50, 90)
cloud_mask = df['clouds'] < 8

mask based on conditions

In [ ]:
masked_df = df[temp_mask & humid_mask & cloud_mask]
# print(len(masked_df))
# check for nulls
masked_df.isna().sum()
len(masked_df)
#%%
# find hotels within 5e3 m of my locations
hotels = []

In [ ]:
for i in masked_df['city']:
    lat = masked_df['lat']
    long = masked_df['long']
    coords = list(zip(lat.astype(str), long.astype(str)))
    coords = [', '.join(coord) for coord in coords]
    param_dict = {
            'location': coords,
            'radius': 5000,
            'type': 'lodging',
            'key': g_key
    }
    url_base = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    try:
        resp = requests.get(url_base, param_dict).json()
        outputs = resp['results'][0]['name']
        hotels.append(outputs)
    except:
        hotels.append(np.nan)
        print(f'Unable to find {i}.')

%%

hotels_found = [resp['results'][i]['name'] for i in range(len(resp['results']))]<br>
print(hotels_found)

In [ ]:
hotel_names = [outputs[i]['name'] for i in range(len(masked_df.city))]
print(len(hotel_names) == len(masked_df.city))
masked_df['hotel'] = hotel_names

%%

In [ ]:
masked_df.reset_index(drop=True)
# %%
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in masked_df.iterrows()]
locations = masked_df[["lat", "long"]]
# %%

In [ ]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# %%